In [1]:
import pandas as pd
from ktrain import text
from transformers import AutoTokenizer, AutoConfig, TFAutoModelForSequenceClassification
from transformers import pipeline
import fasttext
import nltk
import time

In [2]:
ts = text.TransformerSummarizer()
ts2 = pipeline("summarization", model="t5-large", tokenizer="t5-large", framework="tf")
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased', use_fast=True)
model = TFAutoModelForSequenceClassification.from_pretrained("model")
model_ft = fasttext.load_model("fasttext/model.bin")

In [12]:
def predict(text):
    return model(tokenizer.encode(text, return_tensors='tf', max_length=512))[0].numpy()[0][0]

def predict_ft(text):
    return model_ft.predict(" ".join(nltk.word_tokenize(text)))

def print_and_summarize(row):
    text = row['text']
    stars = row['stars']
    print("ORIGINAL TEXT")
    print("\n")
    print(text)
    print("\n")
    print("-------------------------------------------------------------")
    print("\n")
    print("summarizing...")
    start = time.time()
    summary = ts.summarize(text)
    print(f"done. took {time.time() - start} seconds")
    print("\n")
    print("SUMMARY")
    print("\n")
    print(summary)
    print("\n")
    print("-------------------------------------------------------------")
    print("\n")
    print("summarizing 2...")
    start = time.time()
    summary2 = ts2(text, min_length=42, clean_up_tokenization_spaces=True, max_length=1024)[0]['summary_text']
    print(f"done. took {time.time() - start} seconds")
    print("\n")
    print("SUMMARY 2")
    print("\n")
    print(summary2)
    print("\n")
    print("-------------------------------------------------------------")
    print("\n")
    print("original star", stars)
    print("\n")
    print("original pred", predict(text))
    print("summary pred", predict(summary))
    print("summary2 pred", predict(summary2))
    print("original ft pred", predict_ft(text))
    print("summary ft pred", predict_ft(summary))
    print("summary2 ft pred", predict_ft(summary2))

In [13]:
df = pd.read_csv("preds.csv", header=None, names=['idx', 'text', 'stars', 'pred'])
df.shape

(68574, 4)

In [14]:
df.head()

,idx,text,stars,pred
0,0,Really great local auto repair spot... really ...,5.0,4.891084
1,1,Best Oyster House in Toronto. Great variety a...,5.0,5.034161
2,2,We thoroughly enjoy eating here. We go here at...,5.0,4.932011
3,3,OMG!!! So good!!!! I was in AZ for work and I...,5.0,5.016096
4,4,We are here once over a year ago before we mov...,1.0,2.771281


In [15]:
df.isna().sum()

idx      0
text     0
stars    0
pred     0
dtype: int64

In [16]:
df['abs_err'] = abs(df['stars'] - df['pred'])

In [17]:
print_and_summarize(df.sample(1).iloc[0])

ORIGINAL TEXT


It's 4.35

It's overall very good(but it wasn't amazing that I can give it 4.5 or above.  It was very very close though).   All dishes were good but there wasn't a single dish that wowed us(Not a single dish that I would come back for alone).  Service was good.

We had the regular tasting menu ($95 each).  It's a 5 course meal but it's more like a 10 course meal because they provided a bunch of amuse bouche that's not listed on the menu.  Overall it was very good value for the amount of food that you get(at least for this type of restaurant).

They offered complimentary distilled or sparkling water which was a nice gesture for this type of restaurant.

We are not drinkers so we got 2 non-alcoholic drinks:

-Pear Tea and lemon ($8): Good mixture of pear tea and lemon taste.  Rating: 4/5
-Orange and Almond ($8):  Could taste the orange but lacked almond taste. Rating: 3/5

 For the tasting menu we had:

-To begin(don't know exactly what this is called as they didn't put i

In [18]:
print_and_summarize(df.sample(1).iloc[0])

ORIGINAL TEXT


Probably the best Italian restaraunt I've ever been to. Literally AMAZING. The owner came up and talked to us a few times, my water was constantly filled, if the waiter didn't know about a specific wine he woukd ask the manager and he personally came to our table to give us additional information about Italian wine. Amazing shrimp & lobster raviolis. I wish I had more!!! 5 stars forsure.


-------------------------------------------------------------


summarizing...
done. took 25.42374610900879 seconds


SUMMARY


The owner came up and talked to us a few times, my water was constantly filled, if the waiter didn't know about a specific wine he woukd ask the manager and he personally came to our table to give us additional information about Italian wine. Amazing shrimp & lobster raviolis. 5 stars forsure.


-------------------------------------------------------------


summarizing 2...
done. took 137.76110315322876 seconds


SUMMARY 2


Probably the best italian restara

In [19]:
print_and_summarize(df.sample(1).iloc[0])

ORIGINAL TEXT


A typical Vietnamese place. Horrible service, average food, sometimes horrible.
Sriracha sauce to the rescue!

Once I found a live caterpillar in my fresh roll. Another time a whole bunch of hair in my noodles. You get sick sometimes if you eat here regularly. 
As I said, a typical Vietnamese place.


-------------------------------------------------------------


summarizing...
done. took 24.2817280292511 seconds


SUMMARY


Sriracha sauce to the rescue! Once I found a live caterpillar in my fresh roll. Another time a whole bunch of hair in my noodles. You get sick sometimes if you eat here regularly. A typical Vietnamese place. Horrible service, average food, sometimes horrible.


-------------------------------------------------------------


summarizing 2...
done. took 70.2481381893158 seconds


SUMMARY 2


a typical Vietnamese place. Horrible service, average food, sometimes horrible. once i found a live caterpillar in my fresh roll. another time a whole bunch of h

In [20]:
print_and_summarize(df.sample(1).iloc[0])

ORIGINAL TEXT


These guys came out on short notice, did not rip me off! They replaced my pool timer, did not try to up charge me, they did not try to make me pay for more services than what I needed. I will call them again in a hot minute if I ever need a pool guy.


-------------------------------------------------------------


summarizing...
done. took 21.51083278656006 seconds


SUMMARY


These guys came out on short notice, did not rip me off! They replaced my pool timer and did not try to up charge me. I will call them again in a hot minute if I ever need a pool guy. They did not make me pay for more services than what I needed.


-------------------------------------------------------------


summarizing 2...
done. took 68.79211378097534 seconds


SUMMARY 2


they replaced my pool timer, did not try to up charge me. i will call them again in a hot minute if i ever need a pool guy. they came out on short notice.


-------------------------------------------------------------


o

In [ ]:
print_and_summarize(df.sample(1).iloc[0])

ORIGINAL TEXT


I had the porcini mushroom wild rice soup with truffle oil ($10), which came with a cheddar biscuit and a little pocket of sauce on the side. The soup was perfectly fine (I think truffle oil will be one of those acquired taste kinds of items for me) and included a generous portion. 

The biscuit was delectable! You must try it if you're generally a biscuit fan.


-------------------------------------------------------------


summarizing...
done. took 20.57575011253357 seconds


SUMMARY


I had the porcini mushroom wild rice soup with truffle oil. It came with a cheddar biscuit and a little pocket of sauce on the side. The biscuit was delectable! You must try it if you're generally a biscuit fan.


-------------------------------------------------------------


summarizing 2...


In [ ]:
print_and_summarize(df.sample(1).iloc[0])

In [ ]:
print_and_summarize(df.sample(1).iloc[0])

In [ ]:
df['abs_err'].describe()

In [ ]:
top_losses = df[df['abs_err'] > 0.48]
top_losses.shape

In [ ]:
len(top_losses) / len(df)

In [ ]:
top_losses['stars'].value_counts().sort_values(ascending=False)

In [ ]:
top_losses['stars'].plot.hist();

In [ ]:
df['stars'].plot.hist();

In [ ]:
top_losses.sort_values('abs_err').head(10)

In [ ]:
top_losses = df[df['abs_err'] > 1]
top_losses.shape

In [ ]:
len(top_losses) / len(df)

In [ ]:
top_losses['stars'].plot.hist();

In [ ]:
top_losses.sort_values('abs_err').head(10)

In [ ]:
top_losses = df[df['abs_err'] > 1.5]
top_losses.shape

In [ ]:
len(top_losses) / len(df)

In [ ]:
top_losses['stars'].plot.hist();

In [ ]:
top_losses.sort_values('abs_err').head(10)

In [ ]:
print_and_summarize(df.iloc[32127])

In [ ]:
print_and_summarize(df.iloc[25782])

In [ ]:
print_and_summarize(df.iloc[31287])

In [ ]:
print_and_summarize(df.iloc[27611])

In [ ]:
top_top_losses = top_losses[top_losses['abs_err'] > 2]

In [ ]:
top_top_losses.shape

In [ ]:
top_top_losses['stars'].plot.hist();